# SIDA: Speaker Identification for Archives

In [1]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

!mkdir -p /sharedfolder/sida_classifier/training_set/

os.chdir('/sharedfolder/sida_classifier/training_set/')

## Download audio files for training
#### (You may want to comment out the lines below once the download is complete.)
!wget -N http://xtra.arloproject.com/datasets/audio/The_World_WGBH_100_episodes.zip
!unzip The_World_WGBH_100_episodes.zip

training_audio_dir_name = "The_World_WGBH_100_episodes"

--2017-08-24 18:29:38--  http://xtra.arloproject.com/datasets/audio/The_World_WGBH_100_episodes.zip
Resolving xtra.arloproject.com (xtra.arloproject.com)... 146.6.168.12
Connecting to xtra.arloproject.com (xtra.arloproject.com)|146.6.168.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3533087206 (3.3G) [application/zip]
Saving to: 'The_World_WGBH_100_episodes.zip'

100%[====================================>] 3,533,087,206 12.0MB/s   in 4m 41s 

2017-08-24 18:34:19 (12.0 MB/s) - 'The_World_WGBH_100_episodes.zip' saved [3533087206/3533087206]

Archive:  The_World_WGBH_100_episodes.zip
   creating: The_World_WGBH_100_episodes/
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-028pc2tm3q__8_Audio_Track_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-0c4sj1b28r__20130905TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-125q81547n__20131021tw_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-1r6n01063k__20140317tw_.mp3  
 

  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-x05x63bw3b__20140320TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-x921c1vb6n__20130115TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-xk84j0bs8g__20131003tw_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-xw47p8v81r__20140114TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-z31ng4hj4d__10_Audio_Track_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-zg6g15v54b__20131015TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-zs2k64bm8t__20131230TW_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-15-zw18k75s0w__20130924TW6pm_.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-500-9w091844.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-500-9w091883.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-500-bz619d3s.mp3  
  inflating: The_World_WGBH_100_episodes/cpb-aacip-500-k9316q01.mp3  
  inflating: The_World_WGBH_100_episodes/

In [2]:
## Download new 1-second labels

csv_url = "https://raw.githubusercontent.com/hipstas/aapb-speaker-labels/master/speaker_labels_randomized/The_World_WGBH_labels_100_episodes.csv"

csv_string = urllib2.urlopen(csv_url)

train_table = []


## Load CSV as list of lists

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:10] + ['...']

[[u'Media file basename', u'Start time', u'Duration', u'Label', u'Labeled by'],
 [u'19990107_fa_01', u'1171', u'1', u'Male', u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-zg6g15v54b__20131015TW_',
  u'3212',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-500-r785p23b',
  u'346',
  u'1',
  u'Female',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-9jq0st8f__240747_20130619TW',
  u'451',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-ns0ks6jt4h__20130117TW_',
  u'2127',
  u'1',
  u'Female',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-3n20c4t30z__20130925TW_',
  u'294',
  u'1',
  u'Female',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-qf8jd4qc9f__20131219TW_',
  u'2562',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-9vx0627z__236982_20130521TW',
  u'1403',
  u'1',
  u'Female',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-js9h41k82z__20140313TW_',
  u'1652',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 '...']

In [3]:
## Remove header row (if present)

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

In [12]:
%%capture
## Excerpting WAV clips corresponding to labels
#### (This may take a while.)
os.chdir('/sharedfolder/sida_classifier/')

os.chdir('/sharedfolder/sida_classifier/')

media_dir_pathname = "/sharedfolder/sida_classifier/training_set/" + training_audio_dir_name

class_dir_pathname = '/sharedfolder/sida_classifier/training_set/_classes_' + training_audio_dir_name

for row in train_table[3]:
    try:
        basename, start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.mp3')
        start = float(start)
        end = float(start) + float(duration)
        wav_out_pathname = str(os.path.join(class_dir_pathname, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', wav_out_pathname])
        except:
            pass
        attk.subclip(os.path.join(media_dir, filename), float(start), end, wav_out_pathname)
    except Exception as e: 
        print(row)
        print(e)

In [13]:
## Defining functions we will use use below

# Extract audio segments to WAV for an audio/video pathname and a list of 2-tuple time values
def extract_vowel_pairs(media_path, vowel_time_ranges):
    snd = AudioFileClip.AudioFileClip(media_path)
    file_duration = attk.duration(media_path)
    for pair in vowel_time_ranges:
        start, end = pair
        start = float(start)
        end = float(end)
        if end - start >= 0.05:  ## Ignore clips shorter than 0.05 second
            basename = media_path.split('/')[-1][:-4]
            out_filename = basename+'__'+str(round(start, 4))+'_'+str(round(end, 4))+'.wav'
            out_pathname = os.path.join('_vowel_clips',out_filename)
            if not os.path.isfile(out_pathname):
                snd.subclip(start, end).write_audiofile(out_pathname)

# Extract vowel segments to WAV for every audio/video file in a given directory
def batch_extract_vowels(media_dir):
    starting_location = os.getcwd()
    os.chdir(media_dir)
    bin_2048_to_sec_constant = 0.046439909297052155
    try: os.mkdir('_vowel_clips')
    except: pass
    filenames=[item for item in os.listdir('./') if item[-4:].lower() in ('.mp3','.wav','.mp4')]
    for filename in filenames[::-1]:
        try:
            vowel_bools = attk.get_vowel_segments(filename)
            vowel_bin_ranges = attk.labels_to_ranges(vowel_bools, label=True)
            vowel_time_ranges = [(s*bin_2048_to_sec_constant, e*bin_2048_to_sec_constant) for s, e in vowel_bin_ranges]
            extract_vowel_pairs(filename,vowel_time_ranges)
        except Exception as e:
            print("ERROR: " + filename)
            print(e)
    os.chdir(starting_location)

In [14]:
%%capture
## Extract vowel segments from labeled audio clips
#### (This may take a while.)

os.chdir(class_dir_pathname)

batch_extract_vowels('Marco_Werman')
batch_extract_vowels('Male')
batch_extract_vowels('Female')
batch_extract_vowels('Carol_Hills')

In [17]:
#%%capture
## Extract features (MFCCs, deltas, and delta-deltas) from Marco Werman & UBM vowel clips, then write features to CSVs

os.chdir(class_dir_pathname)

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name, '_vowel_clips'))
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

> Starting Female
> Starting Male
> Starting Marco_Werman
> Starting Music
SKIPPING DIRECTORY: Music
[Errno 2] No such file or directory: '/sharedfolder/sida_classifier/training_set/_classes_The_World_WGBH_100_episodes/Music/_vowel_clips'


In [ ]:
%%capture

## Download and unzip prepared UBM feature set

os.chdir('/sharedfolder/sida_classifier/')

!wget -q https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_female_vowel_mfccs_and_deltas.zip?raw=true -O AAPB_female_vowel_mfccs_and_deltas.zip
!wget -q https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_male_vowel_mfccs_and_deltas.zip?raw=true -O AAPB_male_vowel_mfccs_and_deltas.zip

!unzip AAPB_female_vowel_mfccs_and_deltas.zip
!unzip AAPB_male_vowel_mfccs_and_deltas.zip

In [ ]:
## Continue to the next notebook to train and run the speaker ID classifier.